In [1]:
import os
import json
import pandas as pd
import librosa
import numpy as np

# 📂 JSON과 WAV 파일이 저장된 디렉토리 설정
json_dir = "/home/ubuntu/data/test_data/label_data_test"
wav_dir = "/home/ubuntu/data/test_data/raw_data_test"

json_data = []
# JSON 폴더 순회 (예: 1.Car, 2.Motorcycle 등)
for category in os.listdir(json_dir):
    category_path = os.path.join(json_dir, category)

    if os.path.isdir(category_path):  # 폴더인 경우
        for sub_dir in os.listdir(category_path):  # 예: 1.horn_of_car, 2.siren_of_car 등
            sub_path = os.path.join(category_path, sub_dir)

            if os.path.isdir(sub_path):  # 폴더인 경우
                for file in os.listdir(sub_path):  # 폴더 내부 JSON 파일 확인
                    if file.endswith(".json"):
                        file_path = os.path.join(sub_path, file)

                        with open(file_path, "r", encoding="utf-8") as f:
                            data = json.load(f)

                            annotations = data.get("annotations", [])
                            annotation = annotations[0] if annotations and isinstance(annotations, list) else {}

                            # 확장자 제거한 baseName 생성
                            base_name = os.path.splitext(file)[0]

                            json_data.append({
                                "baseName": base_name,  # 파일명에서 확장자 제거
                                "fileName": data["audio"]["fileName"],
                                "labelName": annotation.get("labelName", None),
                                "area_start": annotation.get("area", {}).get("start", None),
                                "area_end": annotation.get("area", {}).get("end", None),
                                "category_01": annotation.get("categories", {}).get("category_01", None),
                                "category_02": annotation.get("categories", {}).get("category_02", None),
                                "category_03": annotation.get("categories", {}).get("category_03", None),
                                "decibel": annotation.get("decibel", None),
                                "soundQuality": annotation.get("soundQuality", None),
                                "subCategory": annotation.get("subCategory", None),
                            })

df_json = pd.DataFrame(json_data)

# 🎵 WAV 파일 목록 가져오기
wav_files = []
for root, _, files in os.walk(wav_dir):
    for file in files:
        if file.endswith(".wav"):
            base_name = os.path.splitext(file)[0]  # 확장자 제거한 파일명
            # 상대 경로로 올바른 폴더 구조 생성
            rel_path = os.path.relpath(root, wav_dir)  # 현재 디렉토리에서 wav_dir까지의 상대 경로
            wav_files.append({"baseName": base_name, "wavFile": os.path.join(rel_path, file)})

df_wav = pd.DataFrame(wav_files)

# df_wav에서 baseName의 마지막 두 글자가 '_1'인 경우만 '_1'을 제거
df_wav['baseName'] = df_wav['baseName'].apply(lambda x: x[:-2] if x.endswith('_1') else x)

# 📌 JSON과 WAV 데이터를 baseName 기준으로 병합
test_merged_df = df_json.merge(df_wav, on="baseName", how="inner")

# 📌 WAV 데이터에서 MFCC 추출하는 함수
def extract_mfcc(file_path, sr=22050, n_mfcc=50):
    try:
        full_file_path = os.path.join(wav_dir, file_path)  # 전체 경로를 생성
        #print(f"파일 경로: {full_file_path}")  # 경로 확인용
        y, sr = librosa.load(full_file_path, sr=sr)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        return np.mean(mfccs, axis=1)  # MFCC 평균값 반환
    except Exception as e:
        print(f"오류 발생: {full_file_path} - {e}")
        return [None] * n_mfcc  # 오류 발생 시 None 리스트 반환

# 📌 MFCC를 각 WAV 파일에 대해 추출하여 데이터프레임에 추가
mfcc_features = []
for index, row in test_merged_df.iterrows():
    wav_file_path = row['wavFile']  # wav 파일 경로
    mfcc = extract_mfcc(wav_file_path)
    mfcc_features.append(mfcc)

# MFCC 결과를 데이터프레임에 추가
mfcc_columns = [f"mfcc_{i+1}" for i in range(len(mfcc_features[0]))]  # MFCC 컬럼명 생성
mfcc_df = pd.DataFrame(mfcc_features, columns=mfcc_columns)
test_merged_df = pd.concat([test_merged_df, mfcc_df], axis=1)

# ✅ 최종 데이터 확인
print(test_merged_df.head())


                  baseName           fileName            labelName  \
0  2.motorcycle_horn_39554  2.이륜자동차_39554.wav  2.이륜자동차_39554_1.wav   
1  2.motorcycle_horn_83873  2.이륜자동차_83873.wav  2.이륜자동차_83873_1.wav   
2  2.motorcycle_horn_83258  2.이륜자동차_83258.wav  2.이륜자동차_83258_1.wav   
3  2.motorcycle_horn_83833  2.이륜자동차_83833.wav  2.이륜자동차_83833_1.wav   
4  2.motorcycle_horn_84352  2.이륜자동차_84352.wav  2.이륜자동차_84352_1.wav   

   area_start  area_end category_01 category_02 category_03  decibel  \
0         2.0     6.819        교통소음       이륜자동차       이륜차경적       74   
1         2.0    15.869        교통소음       이륜자동차       이륜차경적       70   
2         2.0     3.709        교통소음       이륜자동차       이륜차경적       72   
3         2.0    11.257        교통소음       이륜자동차       이륜차경적       74   
4         2.0     3.713        교통소음       이륜자동차       이륜차경적       70   

  soundQuality  ...    mfcc_41    mfcc_42    mfcc_43    mfcc_44    mfcc_45  \
0           정상  ...   8.212692  -4.352332  -2.059963  10.885640  -1.

In [2]:
# CSV 파일로 저장
#test_merged_df.to_csv('model_test_merged_data.csv', index=False)